<a href="https://colab.research.google.com/github/MrKelv/full-tensorflow-rock-paper-scissors/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info

print(info)

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    version=3.0.0,
    description='Images of hands playing rock, paper, scissor game.',
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=2892,
    splits={
        'test': 372,
        'train': 2520,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",
    redistribution_info=,
)



In [3]:
print(f"Data Shape : {info.features['image'].shape}")
print(f"Classifier : {info.features['label'].num_classes}")

Data Shape : (300, 300, 3)
Classifier : 3


In [4]:
print(f"Banyak Data Train : {info.splits['train'].num_examples}")
print(f"Banyak Data Test : {info.splits['test'].num_examples}")

Banyak Data Train : 2520
Banyak Data Test : 372
